This Notebook helps in improving quality of created news data set by adding SBERT based similarity score for each of the similar/related headline.



*   SBERT (Sentence Embeddings using Siamese BERT-Networks) is used for transforming reference headline and similar headline in to their vector representation and then cosine similarity is used for calculating similarity score for similar/related headline against the reference headline.

*   This similarity score will be used for filtering out less relevant similar/related headlines and will help in getting quality news dataset.



In [ ]:
'''
mount google drive folder
'''
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#SBERT for sentance level similarity analysis 
#https://github.com/UKPLab/sentence-transformers
#https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 81kB 5.7MB/s 
     |████████████████████████████████| 2.1MB 11.4MB/s 
     |████████████████████████████████| 1.2MB 44.7MB/s 
     |████████████████████████████████| 3.3MB 48.5MB/s 
     |████████████████████████████████| 870kB 46.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=2d22cd0392dc7ac165b8c563cadb50fd95eea4896ad2b23364fadd63f8a612b7
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b6927fe0b223308c7faed8d069d24f11e46e3f293f482c9e8af809384e2a6d61
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [ ]:
'''
import required packages
'''
import glob
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer, util

In [ ]:
'''
list the saved news data pickle file. (news-url,ref-headline,news-content,author,publish-date,similar-headline-list,similar-headline-url-list)
'''
base_data_loc='/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/'
data_file_list=glob.glob(base_data_loc+'*')
data_file_list

['/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_1_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_2_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_3_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_4_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_5_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_6_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_7_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_8_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/data_9_50.df',
 '/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_a

In [ ]:
individual_data_file_df_list=[pd.read_pickle(data_file) for data_file in data_file_list]
article_details_df = pd.concat(individual_data_file_df_list, axis=0, ignore_index=True)
print('article_details_df shape : ',article_details_df.shape)

article_details_df shape :  (3000, 8)


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [ ]:
article_details_df.head(5)

,article_url,headline,content,author,published_date,read_more_source,similar_headline,similar_headline_url
0,https://inshorts.com/en/news/you-shall-always-have-my-support-arjun-kapoor-on-janhvis-bday-1615047490059,You shall always have my support: Arjun Kapoor on Janhvi's b'day,"Taking to Instagram on Saturday, Arjun Kapoor posted a picture of himself with Janhvi Kapoor to wish the actress on her 24th birthday. In the picture, Arjun can be seen walking ahead while holding his sister's hand. ""Happy birthday Janhvi...I can't promise much except like this picture you shall always have my support & hand wherever you go,"" Arjun wrote.",None,2021-03-06T16:18:10.000Z,PINKVILLA,"[On Janhvi Kapoor’s birthday, Arjun promises to be a constant support: You shall have my hand wherever you go, ""You will always have my support,"" Arjun Kapoor writes a heartfelt birthday note for Janhvi Kapoor., 'You shall always have my support': Arjun Kapoor pens heart-warming birthday note for Janhvi, Janhvi Kapoor rings in 24th birthday with her crew, sister Anshula plans huge surprise]","[https://www.pinkvilla.com/entertainment/news/janhvi-kapoor-s-birthday-arjun-promises-be-constant-support-you-shall-have-my-hand-wherever-you-go-633968, https://indianewsrepublic.com/you-will-always-have-my-support-arjun-kapoor-writes-a-heartfelt-birthday-note-for-janhvi-kapoor/222825/, https://english.lokmat.com/entertainment/you-shall-always-have-my-support-arjun-kapoor-pens-heart-warming-birthday-note-for-janhvi/, https://www.republicworld.com/entertainment-news/bollywood-news/janhvi-kapoor-rings-in-24th-birthday-with-her-crew-sister-anshula-plans-huge-surprise.html]"
1,https://inshorts.com/en/news/gavaskar-felicitated-on-50th-anniversary-of-test-debut-1615046996400,Gavaskar felicitated on 50th anniversary of Test debut,"Ex-India captain Sunil Gavaskar was felicitated by the BCCI on the 50th anniversary of his Test debut on Saturday. The 71-year-old received a commemorative Test cap from BCCI Secretary Jay Shah, while several ex-cricketers took to Twitter to congratulate Gavaskar. ""The game is as strong as it is today because they made a start then against all odds,"" Ganguly tweeted.",None,2021-03-06T16:09:56.000Z,read more source not available,"[Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated by BCCI on 50th anniversary of his Test debut, Former India captain Sunil Gavaskar felicitated by BCCI on 50th anniversary of Test debut, Sunil Gavaskar Felicitated By BCCI On 50th Anniversary Of Test Debut. Watch, Sunil Gavaskar felicitated by BCCI on 50th anniversary of Test debut, BCCI Felicitates Sunil Gavaskar On His 50th Anniversary Of Test Debut]","[https://www.thehindu.com/sport/cricket/sunil-gavaskar-felicitated-on-50th-anniversary-of-test-debut/article34005053.ece, https://www.deccanherald.com/sports/cricket/sunil-gavaskar-felicitated-on-50th-anniversary-of-test-debut-958854.html, https://timesofindia.indiatimes.com/sports/cricket/england-in-india/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut/articleshow/81362569.cms, https://www.sportskeeda.com/cricket/sunil-gavaskar-felicitated-by-bcci-50th-anniversary-test-debut, https://m.economictimes.com/news/sports/former-india-captain-sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut/articleshow/81362570.cms, https://sports.ndtv.com/cricket/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut-watch-2384926, https://www.indiatvnews.com/sports/cricket/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut-689160, https://cricketaddictor.com/cricket-news/bcci-felicitates-sunil-gavaskar-on-his-50th-anniversary-of-test-debut/]"
2,https://inshorts.com/en/news/pv-sindhu-to-face-carolina-marin-in-her-1st-final-since-2019-world-cships-win-1615046535615,PV Sindhu to face Carolina Marin in her 1st fin

In [ ]:
'''
load the pre-trained SBERT model. this will be used for finding similarity between two sentences. 
'''
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:
'''
function to calculate similarity score for each of the similar headline against reference headline
'''
def get_similarity_scores(reference_headline,similar_reference_headlines):
  cosine_scores=[]
  #Compute embedding for both lists
  ref_headline_embed = model.encode(reference_headline, convert_to_tensor=True)
  for headline in similar_reference_headlines:
    headline_embed = model.encode(headline, convert_to_tensor=True)
    #print('embedding shape: ',headline_embed.shape)
    #Compute cosine-similarits
    cosine_score = util.pytorch_cos_sim(ref_headline_embed, headline_embed).item()
    cosine_scores.append(round(cosine_score, 2))
  return cosine_scores

In [ ]:
'''
test function 'get_similarity_scores'
'''
news_summary='A 58-year-old suspected COVID-19 patient, who was put on ventilator, was wrongly declared dead twice by authorities earlier this week at a hospital in Madhya Pradesh. The second time the patient\'s family even prepared for his funeral rites, only for the hospital officials to admit the mix-up shortly after. Officials at the hospital attributed the incident to \"confusion\".'
reference_headline='Suspected COVID-19 patient wrongly declared dead twice by MP hospital(Inshorts)'
similar_reference_headlines=['Madhya Pradesh Hospital Wrongly Declares Covid Patient Dead - Twice',
'In major goof-up, Madhya Pradesh man on ventilator declared dead twice',
'MP COVID mess: Patient declared dead twice by hospital; mismatch at official death figures']
print(get_similarity_scores(reference_headline,similar_reference_headlines))

df_row=article_details_df.iloc[1]
reference_headline=df_row['headline']
similar_reference_headlines=df_row['similar_headline']
print('reference_headline: ',reference_headline)
print('similar_reference_headlines: ',similar_reference_headlines)
print('similarity scores: ',get_similarity_scores(reference_headline,similar_reference_headlines))

[0.59, 0.44, 0.79]
reference_headline:  Gavaskar felicitated on 50th anniversary of Test debut
similar_reference_headlines:  ['Sunil Gavaskar felicitated on 50th anniversary of Test debut', 'Sunil Gavaskar felicitated on 50th anniversary of Test debut', 'Sunil Gavaskar felicitated on 50th anniversary of Test debut', 'Sunil Gavaskar felicitated by BCCI on 50th anniversary of his Test debut', 'Former India captain Sunil Gavaskar felicitated by BCCI on 50th anniversary of Test debut', 'Sunil Gavaskar Felicitated By BCCI On 50th Anniversary Of Test Debut. Watch', 'Sunil Gavaskar felicitated by BCCI on 50th anniversary of Test debut', 'BCCI Felicitates Sunil Gavaskar On His 50th Anniversary Of Test Debut']
similarity scores:  [0.95, 0.95, 0.95, 0.88, 0.86, 0.76, 0.88, 0.81]


In [ ]:
'''
calculate similarity score for similar headline against reference headline for evry news data record
'''
similarity_scores=[]
for index, row in article_details_df.iterrows():
  reference_headline=row['headline']
  similar_reference_headlines=row['similar_headline']
  scores=get_similarity_scores(reference_headline,similar_reference_headlines)
  similarity_scores.append(scores)

In [ ]:
'''
add calculated similarity score as new column in the existing news dataset
'''
article_details_df['similarity_scores']=similarity_scores

In [ ]:
'''
show a sample record
'''
article_details_df.head(3)

,article_url,headline,content,author,published_date,read_more_source,similar_headline,similar_headline_url,similarity_scores
0,https://inshorts.com/en/news/you-shall-always-have-my-support-arjun-kapoor-on-janhvis-bday-1615047490059,You shall always have my support: Arjun Kapoor on Janhvi's b'day,"Taking to Instagram on Saturday, Arjun Kapoor posted a picture of himself with Janhvi Kapoor to wish the actress on her 24th birthday. In the picture, Arjun can be seen walking ahead while holding his sister's hand. ""Happy birthday Janhvi...I can't promise much except like this picture you shall always have my support & hand wherever you go,"" Arjun wrote.",None,2021-03-06T16:18:10.000Z,PINKVILLA,"[On Janhvi Kapoor’s birthday, Arjun promises to be a constant support: You shall have my hand wherever you go, ""You will always have my support,"" Arjun Kapoor writes a heartfelt birthday note for Janhvi Kapoor., 'You shall always have my support': Arjun Kapoor pens heart-warming birthday note for Janhvi, Janhvi Kapoor rings in 24th birthday with her crew, sister Anshula plans huge surprise]","[https://www.pinkvilla.com/entertainment/news/janhvi-kapoor-s-birthday-arjun-promises-be-constant-support-you-shall-have-my-hand-wherever-you-go-633968, https://indianewsrepublic.com/you-will-always-have-my-support-arjun-kapoor-writes-a-heartfelt-birthday-note-for-janhvi-kapoor/222825/, https://english.lokmat.com/entertainment/you-shall-always-have-my-support-arjun-kapoor-pens-heart-warming-birthday-note-for-janhvi/, https://www.republicworld.com/entertainment-news/bollywood-news/janhvi-kapoor-rings-in-24th-birthday-with-her-crew-sister-anshula-plans-huge-surprise.html]","[0.7, 0.8, 0.8, 0.41]"
1,https://inshorts.com/en/news/gavaskar-felicitated-on-50th-anniversary-of-test-debut-1615046996400,Gavaskar felicitated on 50th anniversary of Test debut,"Ex-India captain Sunil Gavaskar was felicitated by the BCCI on the 50th anniversary of his Test debut on Saturday. The 71-year-old received a commemorative Test cap from BCCI Secretary Jay Shah, while several ex-cricketers took to Twitter to congratulate Gavaskar. ""The game is as strong as it is today because they made a start then against all odds,"" Ganguly tweeted.",None,2021-03-06T16:09:56.000Z,read more source not available,"[Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated by BCCI on 50th anniversary of his Test debut, Former India captain Sunil Gavaskar felicitated by BCCI on 50th anniversary of Test debut, Sunil Gavaskar Felicitated By BCCI On 50th Anniversary Of Test Debut. Watch, Sunil Gavaskar felicitated by BCCI on 50th anniversary of Test debut, BCCI Felicitates Sunil Gavaskar On His 50th Anniversary Of Test Debut]","[https://www.thehindu.com/sport/cricket/sunil-gavaskar-felicitated-on-50th-anniversary-of-test-debut/article34005053.ece, https://www.deccanherald.com/sports/cricket/sunil-gavaskar-felicitated-on-50th-anniversary-of-test-debut-958854.html, https://timesofindia.indiatimes.com/sports/cricket/england-in-india/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut/articleshow/81362569.cms, https://www.sportskeeda.com/cricket/sunil-gavaskar-felicitated-by-bcci-50th-anniversary-test-debut, https://m.economictimes.com/news/sports/former-india-captain-sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut/articleshow/81362570.cms, https://sports.ndtv.com/cricket/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut-watch-2384926, https://www.indiatvnews.com/sports/cricket/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut-689160, https://cricketaddictor.com/cricket-news/bcci-felicitates-sunil-gavaskar-on-his-50th-anniversary-of-test-debut/]","[0.95, 0.95, 0.95, 0.88, 0.86, 0.76, 0.88, 0.81]"
2,https://inshorts.com/en/news/pv-sindhu-to-face-carolina-marin-in-her-1st-fi

In [ ]:
'''
save the news-data datafame in to pickle file 
'''
article_details_df.to_pickle('/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/news_article_with_sim_score.df')


In [ ]:
'''
load the news dataset pickle file and show one sample record
'''
article_details_df=pd.read_pickle('/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/news_article_with_sim_score.df')
article_details_df.head(3)

,article_url,headline,content,author,published_date,read_more_source,similar_headline,similar_headline_url,similarity_scores
0,https://inshorts.com/en/news/you-shall-always-have-my-support-arjun-kapoor-on-janhvis-bday-1615047490059,You shall always have my support: Arjun Kapoor on Janhvi's b'day,"Taking to Instagram on Saturday, Arjun Kapoor posted a picture of himself with Janhvi Kapoor to wish the actress on her 24th birthday. In the picture, Arjun can be seen walking ahead while holding his sister's hand. ""Happy birthday Janhvi...I can't promise much except like this picture you shall always have my support & hand wherever you go,"" Arjun wrote.",None,2021-03-06T16:18:10.000Z,PINKVILLA,"[On Janhvi Kapoor’s birthday, Arjun promises to be a constant support: You shall have my hand wherever you go, ""You will always have my support,"" Arjun Kapoor writes a heartfelt birthday note for Janhvi Kapoor., 'You shall always have my support': Arjun Kapoor pens heart-warming birthday note for Janhvi, Janhvi Kapoor rings in 24th birthday with her crew, sister Anshula plans huge surprise]","[https://www.pinkvilla.com/entertainment/news/janhvi-kapoor-s-birthday-arjun-promises-be-constant-support-you-shall-have-my-hand-wherever-you-go-633968, https://indianewsrepublic.com/you-will-always-have-my-support-arjun-kapoor-writes-a-heartfelt-birthday-note-for-janhvi-kapoor/222825/, https://english.lokmat.com/entertainment/you-shall-always-have-my-support-arjun-kapoor-pens-heart-warming-birthday-note-for-janhvi/, https://www.republicworld.com/entertainment-news/bollywood-news/janhvi-kapoor-rings-in-24th-birthday-with-her-crew-sister-anshula-plans-huge-surprise.html]","[0.7, 0.8, 0.8, 0.41]"
1,https://inshorts.com/en/news/gavaskar-felicitated-on-50th-anniversary-of-test-debut-1615046996400,Gavaskar felicitated on 50th anniversary of Test debut,"Ex-India captain Sunil Gavaskar was felicitated by the BCCI on the 50th anniversary of his Test debut on Saturday. The 71-year-old received a commemorative Test cap from BCCI Secretary Jay Shah, while several ex-cricketers took to Twitter to congratulate Gavaskar. ""The game is as strong as it is today because they made a start then against all odds,"" Ganguly tweeted.",None,2021-03-06T16:09:56.000Z,read more source not available,"[Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated on 50th anniversary of Test debut, Sunil Gavaskar felicitated by BCCI on 50th anniversary of his Test debut, Former India captain Sunil Gavaskar felicitated by BCCI on 50th anniversary of Test debut, Sunil Gavaskar Felicitated By BCCI On 50th Anniversary Of Test Debut. Watch, Sunil Gavaskar felicitated by BCCI on 50th anniversary of Test debut, BCCI Felicitates Sunil Gavaskar On His 50th Anniversary Of Test Debut]","[https://www.thehindu.com/sport/cricket/sunil-gavaskar-felicitated-on-50th-anniversary-of-test-debut/article34005053.ece, https://www.deccanherald.com/sports/cricket/sunil-gavaskar-felicitated-on-50th-anniversary-of-test-debut-958854.html, https://timesofindia.indiatimes.com/sports/cricket/england-in-india/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut/articleshow/81362569.cms, https://www.sportskeeda.com/cricket/sunil-gavaskar-felicitated-by-bcci-50th-anniversary-test-debut, https://m.economictimes.com/news/sports/former-india-captain-sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut/articleshow/81362570.cms, https://sports.ndtv.com/cricket/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut-watch-2384926, https://www.indiatvnews.com/sports/cricket/sunil-gavaskar-felicitated-by-bcci-on-50th-anniversary-of-test-debut-689160, https://cricketaddictor.com/cricket-news/bcci-felicitates-sunil-gavaskar-on-his-50th-anniversary-of-test-debut/]","[0.95, 0.95, 0.95, 0.88, 0.86, 0.76, 0.88, 0.81]"
2,https://inshorts.com/en/news/pv-sindhu-to-face-carolina-marin-in-her-1st-fi

In [ ]:
'''
save the news-data datafame in to excel file 
'''
article_details_df.to_excel('/content/gdrive/My Drive/capstone_project/news_url_with_headline/News_atricle_details/news_article_with_sim_score.xlsx')
